## Operations

In [70]:
class Operation():
    def __init__(self,input_nodes=[]):
        
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
    def compute(self):
        pass

In [71]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [88]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [73]:
class matmul(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [74]:
class Placeholder():
    
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [75]:
class Variable():
    
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [76]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders =[]
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

z = AX + b

A = 10

b = 1

z = 10x + 1

In [77]:
g = Graph()

In [78]:
g.set_as_default()

In [79]:
A = Variable(10)

In [80]:
b = Variable(1)

In [81]:
x = Placeholder()

In [82]:
y = multiply(A,x)

In [83]:
z = add(y,b)

In [84]:
# traverse Operations

def traverse_postorder(operation):
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder



In [90]:
class Session():
    
    def run(self,operation,feed_dict={}):
        
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == Placeholder:
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else:
                #operations
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
            
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        return operation.output    
        

In [91]:
sess = Session()

In [92]:
result = sess.run(operation=z,feed_dict={x:10})